# Scrape website 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' for all Toronto neighborhoods

import libraries for data analysis (np and pd) and accessing website data (requests and BeautifulSoup)

In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

access website and download all text

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url,'lxml')


Read in main table data to variable My_table with column headers PostCode, Borough, and Neighborhood.  Read by table row and separate by text td.  Drop missing values.  Strip out strings \n

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})
rows  = My_table.find_all('tr')
TN=[]
for tr in rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    #print(row)
    TN.append(row)
df=pd.DataFrame(TN, columns=['PostCode', 'Borough', 'Neighborhood'])
df = df.replace(to_replace='None', value=np.nan).dropna()
df['Neighborhood']=df['Neighborhood'].str.strip('\n')


Drop row if Borough Not assigined.  
If Neighborhood set to Not assigned replace with borough name.  
Group by postcode and borough.  
Assign one row for each postal code, in neighborhood column place all neighborhoods separated by comma.
Place Result into variable TN

In [6]:

df['Borough'] = df['Borough'].replace(to_replace="Not assigned", value=np.nan)
df=df.dropna()

df['Neighborhood'] = df['Neighborhood'].replace(to_replace="Not assigned", value=np.nan)
values=df['Borough']
df['Neighborhood'] = values.where(df['Neighborhood'].isnull(), other=df['Neighborhood'])

TN=df.groupby(['PostCode', 'Borough'], sort=False)['Neighborhood'].apply(','.join)

TN.head(20)

PostCode  Borough         
M3A       North York                                                  Parkwoods
M4A       North York                                           Victoria Village
M5A       Downtown Toronto                             Harbourfront,Regent Park
M6A       North York                            Lawrence Heights,Lawrence Manor
M7A       Queen's Park                                             Queen's Park
M9A       Etobicoke                                            Islington Avenue
M1B       Scarborough                                             Rouge,Malvern
M3B       North York                                            Don Mills North
M4B       East York                              Woodbine Gardens,Parkview Hill
M5B       Downtown Toronto                              Ryerson,Garden District
M6B       North York                                                  Glencairn
M9B       Etobicoke           Cloverdale,Islington,Martin Grove,Princess Gar...
M1C       Sca

Find number of rows of data with .shape

In [11]:
TN.shape

(103,)